https://www.home-assistant.io/installation/linux

In [ ]:
wget https://github.com/home-assistant/operating-system/releases/download/12.4/haos_ova-12.4.qcow2.xz

In [5]:
%pwd

'/home/griffin/git/hot/infrastructure/gcp/homeassistant'

In [12]:
! ls haos_ova-12.4.qcow2

haos_ova-12.4.qcow2


In [ ]:
# loader is instead of ovmf

In [17]:
! virt-install \
    --name homeassistant \
     --os-variant=generic \
     --ram=2048 \
      --vcpus=2 \
       --disk haos_ova-12.4.qcow2,bus=scsi \
       --controller type=scsi,model=virtio-scsi \
       --import \
       --graphics none \
       --boot loader="$(guix build ovmf)/share/firmware/ovmf_x64.bin"

WARNING  Using --osinfo generic, VM performance may suffer. Specify an accurate OS for optimal results.

Starting install...
Creating domain...                                          |    0 B  00:00     
Running text console command: virsh --connect qemu:///session console homeassistant
Connected to domain 'homeassistant'
Escape character is ^] (Ctrl + ])
[=3h[=3hBdsDxe: loading Boot0001 "UEFI QEMU QEMU HARDDISK " from PciRoot(0x0)/Pci(0x3,0x0)/Scsi(0x0,0x0)
BdsDxe: starting Boot0001 "UEFI QEMU QEMU HARDDISK " from PciRoot(0x0)/Pci(0x3,0x0)/Scsi(0x0,0x0)
error: file `/EFI/BOOT/grubenv' not found.
error: file `/EFI/BOOT/grubenv' not found.
GNU GRUB  version 2.12

┌────────────────────────────────────────────────────────────────────────────┐││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││││└────────────────────────────────────────────────────────────────────────────┘     Use the ▲ and ▼ keys to select which entry is highlighted.          
      Press enter t

In [29]:
! qemu-system-x86_64 \
		-enable-kvm \
		-drive "file=haos_ova-12.4.qcow2,format=qcow2" \
		-m 2048 \
		-nic "user,model=virtio-net-pci,hostfwd=tcp::8123-:8123" \
        -bios "$(guix build ovmf)/share/firmware/ovmf_x64.bin"


Shell> bcfg boot add 0 FS0:\EFI\BOOT\grubx64.efi "My GRUB Bootloader"
   Shell> reset

In [23]:
! qemu-system-x86_64 \
		-enable-kvm \
		-drive "if=virtio,file=haos_ova-12.4.qcow2,format=qcow2" \
		-device virtio-scsi-pci,id=scsi \
		-m 2048 \
		-nic "user,model=virtio-net-pci,hostfwd=tcp:127.0.0.1:8123-:8123" \
        -bios "$(guix build ovmf)/share/firmware/ovmf_x64.bin"
